In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

In [2]:
seg_df = pd.read_csv('DATA SET FOR RFM-VALUE BASED SEGMENTATION.csv')

In [3]:
seg_df.head()

,cust_id,No_of_days_frm_L_pur,total_sale,No_baskets,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2
0,347,3,444.00,4,47,25,5,11.75000,0
1,585,320,509871.48,27,19322,531,68,715.62963,0
2,885,115,2284.59,4,247,130,17,61.75000,1
3,1670,3,8092.17,28,553,306,114,19.75000,1
4,2188,428,200.50,1,51,14,2,51.00000,0


In [4]:
pandas_profiling.ProfileReport(seg_df)

Number of variables,9
Number of observations,22602
Total Missing (%),0.0%
Total size in memory,1.6 MiB
Average record size in memory,72.0 B
Numeric,7
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [5]:
seg_df.rename(columns={'No_of_days_frm_L_pur': 'recency', 
                         'No_baskets': 'frequency', 
                         'total_sale': 'monetary'}, inplace=True)

In [6]:
seg_df.columns

Index(['cust_id', 'recency', 'monetary', 'frequency', 'item_cnt', 'no_SKUs',
       'no_Categories', 'avg_no_prds_w_ret', 'Flag_Multi_cat_p2'],
      dtype='object')

### value based segmentation

In [7]:
# creating Deciles
seg_df['decile'] = pd.qcut(seg_df['monetary'], 10, labels=False)

In [8]:
seg_df.head()

,cust_id,recency,monetary,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,decile
0,347,3,444.00,4,47,25,5,11.75000,0,5
1,585,320,509871.48,27,19322,531,68,715.62963,0,9
2,885,115,2284.59,4,247,130,17,61.75000,1,8
3,1670,3,8092.17,28,553,306,114,19.75000,1,9
4,2188,428,200.50,1,51,14,2,51.00000,0,3


In [17]:
seg_df.decile.value_counts()

0    2276
3    2262
9    2261
8    2260
7    2260
6    2260
5    2260
2    2259
4    2258
1    2246
Name: decile, dtype: int64

In [19]:
# Key performace variable selection
col_kpi=['monetary','recency','frequency','item_cnt','no_SKUs',
         'no_Categories','avg_no_prds_w_ret','Flag_Multi_cat_p2']

In [20]:
#Profiling
VB_Profiling=seg_df.groupby(['decile']).apply(lambda x: x[col_kpi].mean())

In [21]:
profiling = VB_Profiling.T

In [22]:
profiling

decile,0,1,2,3,4,5,6,7,8,9
monetary,69.865536,107.423593,153.687809,229.448081,331.000137,474.869996,712.810314,1140.993296,2013.107841,7405.863361
recency,238.693761,197.436331,187.954405,170.429708,174.325066,123.691150,100.781858,77.999115,61.258407,50.987616
frequency,1.019772,1.018255,1.194776,1.683908,2.343667,2.805752,4.040265,6.153540,9.705752,18.102609
item_cnt,6.140598,10.094390,11.041169,18.546861,25.880425,34.734071,50.161504,80.302655,137.839823,354.333481
no_SKUs,3.235501,5.451024,5.681275,9.091954,12.349867,16.421239,25.389381,40.023894,65.577876,129.369306
no_Categories,1.648067,2.228406,2.354139,3.597259,4.984500,6.288938,9.525664,15.100442,24.642035,48.166298
avg_no_prds_w_ret,5.964572,9.995548,9.499277,11.684586,11.650923,14.139388,13.328497,14.260162,15.953250,24.099957
Flag_Multi_cat_p2,0.117311,0.334817,0.326251,0.416004,0.443313,0.588938,0.699558,0.799115,0.846903,0.865104


In [14]:
profiling.to_csv('profiling.csv')

### RFM Segmentation

In [23]:
#Method-2: Creating quartiles

quantiles = seg_df[['frequency','monetary','recency' ]].quantile(q=[0.25,0.5,0.75])
quantiles

,frequency,monetary,recency
0.25,1.0,151.800,26.0
0.50,2.0,393.755,93.0
0.75,5.0,1117.725,225.0


In [24]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'monetary': {0.25: 151.8, 0.5: 393.755, 0.75: 1117.725},
 'recency': {0.25: 26.0, 0.5: 93.0, 0.75: 225.0}}

In [26]:
#We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good.
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [27]:
seg_df['R_Quartile'] = seg_df['recency'].apply(RClass, args=('recency',quantiles,))
seg_df['F_Quartile'] = seg_df['frequency'].apply(FMClass, args=('frequency',quantiles,))
seg_df['M_Quartile'] = seg_df['monetary'].apply(FMClass, args=('monetary',quantiles,))

In [28]:
seg_df.head()

,cust_id,recency,monetary,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,decile,R_Quartile,F_Quartile,M_Quartile
0,347,3,444.00,4,47,25,5,11.75000,0,5,1,2,2
1,585,320,509871.48,27,19322,531,68,715.62963,0,9,4,1,1
2,885,115,2284.59,4,247,130,17,61.75000,1,8,3,2,1
3,1670,3,8092.17,28,553,306,114,19.75000,1,9,1,1,1
4,2188,428,200.50,1,51,14,2,51.00000,0,3,4,4,3


In [29]:
seg_df1=seg_df.assign(R_seg=np.nan, FM_seg =np.nan)
seg_df1.R_seg[seg_df['R_Quartile']==4] = "3.Churn"
seg_df1.R_seg[seg_df['R_Quartile']==2] = "2.At Risk"
seg_df1.R_seg[seg_df['R_Quartile']==3] = "2.At Risk"
seg_df1.R_seg[seg_df['R_Quartile']==1] = "1.Active"

seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==1)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==1)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==2)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==1)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==2)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==2)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==3)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==3)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==3)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==3)] = "0.Standard"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==2)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==4)] = "0.Standard"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==4)] = "0.Standard"

seg_df1.FM_seg.fillna("4.Premium", inplace=True)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\USER\Anaconda3\lib\site

In [21]:
seg_df1.head(5)

,cust_id,recency,monetary,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,decile,R_Quartile,F_Quartile,M_Quartile,R_seg,FM_seg
0,347,3,444.00,4,47,25,5,11.75000,0,5,1,2,2,1.Active,2.Gold
1,585,320,509871.48,27,19322,531,68,715.62963,0,9,4,1,1,3.Churn,4.Premium
2,885,115,2284.59,4,247,130,17,61.75000,1,8,3,2,1,2.At Risk,2.Gold
3,1670,3,8092.17,28,553,306,114,19.75000,1,9,1,1,1,1.Active,4.Premium
4,2188,428,200.50,1,51,14,2,51.00000,0,3,4,4,3,3.Churn,1.Silver


In [30]:
# Key performace variable selection
col_kpi=['monetary','recency','frequency','item_cnt','no_SKUs',
         'no_Categories','avg_no_prds_w_ret','Flag_Multi_cat_p2']

In [31]:
#Profiling
Profiling=seg_df1.groupby(['R_seg', 'FM_seg']).apply(lambda x: x[col_kpi].mean()).T

In [32]:
Profiling

R_seg                1.Active                                       \
FM_seg             0.Standard    1.Silver      2.Gold    4.Premium   
monetary           141.844868  707.378922  946.054019  4600.887834   
recency             12.156085   12.101942   11.724083    10.605064   
frequency            1.022487    3.928155    3.850080    17.181784   
item_cnt             9.370370   46.337864   51.342903   281.465231   
no_SKUs              5.062169   23.479612   24.006380   122.123205   
no_Categories        2.034392    9.309709    8.876396    45.893802   
avg_no_prds_w_ret    9.234127   13.860883   13.301449    15.871176   
Flag_Multi_cat_p2    0.562169    0.861165    0.921850     0.988662   

R_seg               2.At Risk                                        \
FM_seg             0.Standard    1.Silver       2.Gold    4.Premium   
monetary           139.958281  613.077307  1050.597878  3196.114537   
recency            118.988692  104.435336    98.533545    73.346466   
frequency            1.021845    2.837102     3.680085    11.850521   
item_cnt             9.525315   35.442403    53.811441   196.280417   
no_SKUs              4.780776   16.559364    22.504944    81.535921   
no_Categories        1.898998    6.361131     8.293785    30.299537   
avg_no_prds_w_ret    9.421999   14.367780    14.966166    16.388437   
Flag_Multi_cat_p2    0.429196    0.645230     0.759181     0.901506   

R_seg                 3.Churn                                       
FM_seg             0.Standard    1.Silver      2.Gold    4.Premium  
monetary           128.820981  795.197623  999.251206  9816.190248  
recency            348.515587  321.752467  324.635214   279.611570  
frequency            1.020477    2.018092    3.574903     9.859504  
item_cnt             9.632335   29.168586   45.163424   347.842975  
no_SKUs              4.939487   11.172697   16.446498    71.991736  
no_Categories        2.150061    4.603618    7.322957    23.958678  
avg_no_prds_w_ret    9.496333   16.252767   12.971190    24.605255  
Flag_Multi_cat_p2    0.000000    0.000000    0.000000     0.000000

In [25]:
Profiling.to_csv('Profiling_rfm.csv')